In [17]:
import numpy as np
import torch

In [99]:
m = 10
n = 5
K = np.random.rand(m,m,n*n,n*n)
X = np.random.rand(m,m,n,n)
print(K[0,0,0,0])

0.395787093802594


In [3]:
def pair_wise_consistency(i,j,X,n,m):
	s = 0
	for k in range(m):
		s += np.linalg.norm(X[i,j]-X[i,k].dot(X[k,j]))/2/n/m
	return 1-s

print(pair_wise_consistency(3,4,X,n,m))

0.5659939936027523


In [4]:
def affinity_score(i,j,X,K,n):
    scr = X[i,j].T.reshape(1,-1).dot(K[i,j]).dot(X[i,j].T.reshape(-1,1))
    GT = np.eye(n,n)
    denom = GT.T.reshape(1,-1).dot(K[i,j]).dot(GT.T.reshape(-1,1))
    return (scr/denom).item()

print(affinity_score(3,4,X,K,n))

7.4579392663481965


In [8]:
from queue import Queue

q = Queue()
for i in range(m):
    q.put_nowait(i)
q.get_nowait()

0

In [ ]:
def affinity_score_vec(X,K,n):
    scr = X.transpose((0,1,3,2)).squeeze(3).dot(K_ij).dot(X_ij.T.reshape(-1,1))
    GT = np.eye(n,n)
    denom = GT.T.reshape(1,-1).dot(K_ij).dot(GT.T.reshape(-1,1))
    return (scr/denom).item()

In [68]:
X_tensor = torch.from_numpy(X)
K_tensor = torch.from_numpy(K)
XT = X_tensor.transpose(2,3)
vec_XT = XT.reshape(XT.shape[0]*XT.shape[1],1,-1)
vec_X = XT.reshape(XT.shape[0]*XT.shape[1],-1,1)
vec_K = K_tensor.reshape(K.shape[0]*K.shape[1],K.shape[2],K.shape[3])
scr = vec_XT.bmm(vec_K).bmm(vec_X)
print(scr.shape)
GT = torch.eye(n,n,dtype=vec_K.dtype).view(1,n,n).expand((m*m,n,n))
print(GT.shape)
vec_GTT = GT.view(GT.shape[0],1,-1)
vec_GT = GT.view(GT.shape[0],-1,1)
denom = vec_GTT.bmm(vec_K).bmm(vec_GT)
(scr/denom).shape

torch.Size([100, 1, 1])
torch.Size([100, 5, 5])


torch.Size([100, 1, 1])

In [63]:
X_tensor = torch.from_numpy(X)
m1 = X_tensor.view(X.shape[0],X.shape[1],1,n,n).expand(-1,-1,m,-1,-1)
m2 = m1.transpose(0,2)
m3 = m1.transpose(1,2)
s = (m3.reshape(-1,n,n) - m1.reshape(-1,n,n).bmm(m2.reshape(-1,n,n))).view(m,m,m,n,n).norm(dim=(3,4)).sum(1).squeeze(1)
print(s.shape)
1-s/m/n/2

torch.Size([10, 10])


In [82]:
X_opt = X_tensor.reshape(-1,n,n).bmm(X_tensor[:,-1].view(m,1,n,n).expand(-1,m,-1,-1).reshape(-1,n,n)).view(X.shape)
X_opt.shape

torch.Size([10, 10, 5, 5])

In [114]:
aff = (scr/denom).view(m,m)
X[aff>5] = X_opt[aff>5]
M,inds = aff.max(1)
print(aff)
print(inds)
print(aff[range(aff.shape[0]),inds])

tensor([[ 5.6231,  5.1422,  4.9460,  9.6391,  5.7485,  7.5755,  5.8343,  5.7320,
          5.4200,  4.3701],
        [ 6.5801,  6.3625,  7.0989,  6.3113,  8.1418,  5.3988,  5.2281,  4.7327,
          4.5765,  5.7603],
        [ 5.8988,  6.0644,  5.6079,  5.2195,  7.6193,  6.9012,  4.8225,  6.9730,
          5.1039,  3.9485],
        [ 4.3872,  8.3421,  7.4522,  5.2932,  6.8692,  7.9756,  8.1800,  4.4112,
          5.5176,  7.4866],
        [ 4.6262,  4.8060,  3.5543,  4.3065,  5.4598,  5.9138,  3.3933,  6.0735,
          6.5628,  4.8698],
        [ 5.7591,  6.6858,  8.4830,  5.0843,  5.6739,  7.6381,  4.5189,  4.4100,
          7.5868, 10.1609],
        [ 9.9538,  8.4004,  4.6207,  7.3428,  4.3703,  3.5732,  7.8005,  6.1169,
          8.1597,  8.6284],
        [ 6.4709,  3.6471,  6.1337,  4.2844,  7.1860,  4.7008,  7.5667,  6.0715,
          6.0595,  5.8643],
        [ 3.8820,  6.1859,  5.9005,  5.2754,  7.2176,  7.2366,  5.8504,  5.1947,
          5.2487,  7.2604],
        [ 3.8866,  

In [90]:
X_tensor[:,-1].shape

torch.Size([10, 5, 5])

In [110]:
aff.diagonal()*aff[:,-1]

tensor([24.5735, 36.6499, 22.1428, 39.6283, 26.5885, 77.6097, 67.3062, 35.6051,
        38.1078, 50.1550], dtype=torch.float64)

tensor([3.8866, 5.3722, 6.5308, 6.7062, 3.8063, 5.2836, 3.8299, 5.4367, 4.5016,
        7.0820], dtype=torch.float64)